In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Implement The Forward() Method

In [15]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, t):
        # 1: input layer
        t = t
        
        # 2: hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # 3: hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # 4: hidden linear layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # 5: hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # 6: output layer
        t = self.out(t)
        
        return t

## Forward Pass

Turn off PyTorch's gradient feature.

In [16]:
torch.set_grad_enabled(False)

### Get a sample.

In [17]:
import torchvision
from torchvision.transforms import transforms

train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True, 
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)
sample = next(iter(train_set))
image, label = sample

In [18]:
image.shape

torch.Size([1, 28, 28])

### Insert an additional dimension that represents a batch of size 1

In [19]:
image.unsqueeze(dim=0).shape

torch.Size([1, 1, 28, 28])

In [21]:
network = Network()
pred = network(image.unsqueeze(dim=0))
pred

tensor([[ 0.0806,  0.0759, -0.0137,  0.0148, -0.0798,  0.0284,  0.1052, -0.0454,
          0.0278,  0.0826]])

In [22]:
pred.shape

torch.Size([1, 10])

In [23]:
pred.argmax(dim=1)

tensor([6])

In [24]:
label

9

In [25]:
F.softmax(pred, dim=1)

tensor([[0.1053, 0.1048, 0.0958, 0.0986, 0.0897, 0.0999, 0.1079, 0.0928, 0.0999,
         0.1055]])

In [26]:
F.softmax(pred, dim=1).sum()

tensor(1.)